In [ ]:
!pip install dash       ## nainstaluji dash


In [ ]:
import pandas as pd                 ## nainstaluji další knihovny
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px 
import dash
from dash import html, dcc, Input, Output
import threading


In [ ]:
df_covid = pd.read_csv('project_1_python.csv')    ##přidám lista csv


In [ ]:
df_covid.info()                 ##zjistím informace o dokumentu

In [ ]:
df_population = df_covid[['location', 'population']].drop_duplicates()  ##vyberu sloupce location a population


In [ ]:
df_population = df_population.sort_values(by='population', ascending=False).head(15) ##vyberu 15 zemí s nejvyšší populací a smažu duplikace


In [ ]:
plt.figure(figsize=(12,8))    ##nastavím velikost a styl grafu
sns.set_theme(style="whitegrid")
population_chart = sns.barplot(x='location', 
                               y='population', 
                               data=df_population,
                               color='#3F3244')


In [ ]:
population_chart.set_xticklabels(population_chart.get_xticklabels(), rotation=60)  ##nastavím rotaci popisků osy X, abych dosáhla co nejlepší čitelnosti:


In [ ]:
df_population_lifeexp = df_covid[['location', 'continent', 'population', 'life_expectancy']].drop_duplicates() ## z dat vyberu pouze sloupce, které mě zajímají, a smažu duplikáty


In [ ]:
pop_lifeexp_chart = px.scatter(data_frame=df_population_lifeexp,    ##vygeneruji bodový graf
                               x='population', 
                               y='life_expectancy',
                               color='continent',
                               color_discrete_sequence=px.colors.qualitative.Prism,
                               title='Life expectancy vs. population by continent',
                               log_x=True)


In [ ]:
pop_lifeexp_chart


Diagnostikování případů

In [ ]:
df_new_cases = df_covid[['location', 'date', 'new_cases']]   ##vyberu data, která mě zajímají
countries = ['Germany', 'Russia']
df_new_cases = df_new_cases[df_new_cases['location'].isin(countries)]


In [ ]:
ttl_cases_chart = px.line(data_frame=df_new_cases,              ##vyberu spojnicový graf
                          x='date', y='new_cases', 
                          color='location',
                          color_discrete_sequence=px.colors.qualitative.T10,
                          title='COVID-19 - new cases daily in Russia and Germany')


In [ ]:
ttl_cases_chart


In [ ]:
df_map = df_covid[df_covid['date'] == df_covid['date'].max()]    


Mapa Covid 19

In [ ]:
df_map = df_covid[df_covid['date'] == df_covid['date'].max()]   ##vybírám nejčerstvějšínejmńovější data


In [ ]:
df_map['cases_per_person'] = df_map['total_cases'] / df_map['population']   ##vytvořím nový slupec poměr případů k polulací


In [ ]:
my_mapbox_token = 'my_token'                    ##definuji můj API Mapbox token a používám je pro přístup ke stylům
px.set_mapbox_access_token(my_mapbox_token)


In [ ]:
cases_map = px.scatter_mapbox(data_frame=df_map,            ##generuji vizualizaci 
                                  lat='latitude', lon='longitude',
                                  color='continent',
                                  size='cases_per_person',
                                  size_max=20,
                                  hover_data={'location': True, 
                                              'total_cases': True,
                                              'continent': False,
                                              'cases_per_person': False,
                                              'latitude': False,
                                              'longitude': False,
                                              },
                                  zoom=1,
                                  mapbox_style='dark',
                                  title='COVID-19 map - positive cases per person')


In [ ]:
cases_map     


Řídící panel 1

In [ ]:
df_dash_1 = df_covid      ##vytvořím kopii df_covid rámci a přoiřadím je k df_dash_1


In [ ]:
countries_list = list(df_dash_1['location'].drop_duplicates())  ##seznam zemí, které použiji jako hodnoty filtru


In [ ]:
first_dash_app = dash.Dash()        ##vytvořím objekt aplikace a definuji rozvržení dashboardu

first_dash_app.layout = html.Div([
    html.Div([
        html.H1('COVID-19 Tracker')
    ]),                           
    html.P("Choose country:"),
    dcc.Dropdown(
        id='country', 
        options=[{'value': str(country), 'label': country} 
                 for country in countries_list],
        value=countries_list[0]
        ),
    html.Br(),
    html.Div(children=[
        dcc.Graph(id="graph-1", style={'display': 'inline-block', 'width': '48%'}),
        dcc.Graph(id="graph-2", style={'display': 'inline-block', 'width': '48%'})
    ])

])


In [ ]:
@first_dash_app.callback(                           ##Vytvořím první zpětné volání, které použiji k aktualizaci informací v graph-1objektu
    Output(component_id='graph-1', component_property='figure'),
    Input(component_id='country', component_property='value')
)
def generate_first_graph(country):
    df = df_dash_1[df_dash_1['location'] == country]
    fig = px.line(
        data_frame=df,
        x='date',
        y='total_cases',
        title=f'Cumulative number of positive cases in {country}',
        labels={'total_cases': 'Total cases'}
    )
    return fig


In [ ]:
@first_dash_app.callback(                   ##tímto zpětný voláním aktualizuji druhý objekt graph-2
    Output(component_id='graph-2', component_property='figure'),
    Input(component_id='country', component_property='value')
)
def generate_second_graph(country):
    df = df_dash_1[df_dash_1['location'] == country]
    fig = px.line(
        data_frame=df,
        x='date',
        y='total_deaths',
        title=f'Cumulative number of deaths in {country}',
        labels={'total_deaths': 'Total deaths'}
    )
    return fig


In [ ]:
import threading
from dash import Dash

# Definujte funkci pro spuštění serveru
def run_dash_app():
    app = Dash(__name__)
    app.layout = ...
    app.run_server(port=8050)

# Vytvořte vlákno pro spuštění aplikace
thread = threading.Thread(target=run_dash_app)
thread.start()


In [ ]:
thread = threading.Thread(target=first_dash_app.run_server)  ##aplikaci spustím pomocí threading knihovny a output modulu
thread.start()
output.serve_kernel_port_as_window(port=8050)


Řídící panel 2
Z df_covid datového rámce vyberu pouze nejnovější záznamy a záznamy obsahující null hodnoty vyplním nulami. 

In [ ]:
df_dash_2 = df_covid[df_covid['date'] == df_covid['date'].max()].fillna(0)


In [ ]:
##vytvořím si slovník, kam uložím názvy sloupců (metriky) a jejich popisky
metric_dict = {'total_cases': 'Total cases', 
               'total_deaths': 'Total deaths', 
               'total_tests': 'Total tests', 
               'total_vaccinations': 'Total vaccinations', 
               'people_fully_vaccinated': 'Number of fully vaccinated people'}

In [ ]:
##vytvořím filtr, který mi umožní vybrat kontinenty, které budemebudu ukládat do seznamu
continent_list = list(df_dash_2['continent'].drop_duplicates())


In [ ]:
##vytvořím objekt aplikace a definuji rozvržení dashboardu
second_dash_app = dash.Dash()

second_dash_app.layout = html.Div([
    html.Div([
        html.H1('COVID-19 Tracker')
    ]),                           
    html.P("Choose continent:"),
    dcc.Dropdown(
        id='continent', 
        options=[{'value': str(continent), 'label': continent} 
                 for continent in continent_list],
        value=continent_list[0]
        ),
    html.P("Choose metric:"),
    dcc.Dropdown(
        id='metric', 
        options=[{'value': metric, 'label': metric_label} 
                 for metric, metric_label in metric_dict.items()],
        value='total_cases'
        ),       
    html.Br(),
    html.Div(children=[
        dcc.Graph(id="map"),
    ])

])


In [ ]:
##vytvořím zpětné volání, které použiji k aktualizaci informací v map objektu
@second_dash_app.callback(
    Output(component_id='map', component_property='figure'),
    [Input(component_id='continent', component_property='value'),
    Input(component_id='metric', component_property='value')]
)
def generate_covid_map(continent, metric):
    df = df_dash_2[df_dash_2['continent'] == continent]
    map = px.scatter_mapbox(
        data_frame=df,
        lat='latitude', 
        lon='longitude',
        size=df[metric],
        size_max=20,
        hover_data={'location': True},
        mapbox_style='dark',
        zoom=1,
        height=600,
        title=f'COVID-19 - {metric_dict[metric]} in {continent}'
    )
    return map


In [ ]:
##xaplikaci spustím metodou .run_server() pomocí threading knihovny a outputmodulu
thread = threading.Thread(target=second_dash_app.run_server)
thread.start()
output.serve_kernel_port_as_window(port=8050)


Řídící panel č. 3

In [70]:
##vytvořím aplikační objekt  aplikačního objektu a definuji rozvržení dashboardu 
third_dash_app = dash.Dash()

third_dash_app.layout = html.Div([
    html.Div([
        html.H1('COVID-19 Tracker')
    ]),                           
    html.P("Choose top x countries:"),
    dcc.Slider(5, 20, 5,
        value=5,
        id='my-slider'
    ),   
    html.Br(),
    html.Div(children=[
        dcc.Graph(id='first-graph', style={'display':'inline-block', 'width': '48%'}),
        dcc.Graph(id='second-graph', style={'display':'inline-block', 'width': '48%'})
    ])

])


In [71]:
##vytvořím zpětné volání, které použiji k aktualizaci informací v first_graphobjektu. 
##umsím vybrat nejčerstvější záznamy a seřadit země podle počtu očkování a vybrat pouze n záznamy. Hodnoty null nahradím nulami
@third_dash_app.callback(
    Output(component_id='first-graph', component_property='figure'),
    [Input(component_id='my-slider', component_property='value')]
)
def generate_vaccinations_graph(n):
    max_date = df_covid['date'].max()
    df = df_covid[df_covid['date'] == max_date].sort_values(by='total_vaccinations', ascending=False).head(n).fillna(0)
    fig = px.bar(
        data_frame=df,
        x='location',
        y='total_vaccinations',
        title=f'Number of vaccinations',
        labels={'total_vaccinations': 'Total vaccinations'}
    )
    return fig


In [72]:
##druhé zpětné volání předá figuru second-graph objektu. 
##vytvořím nové vaccinaton_ratio měřítko, které vypočítá poměr celkového počtu proočkovaných osob k populaci
@third_dash_app.callback(
    Output(component_id='second-graph', component_property='figure'),
    [Input(component_id='my-slider', component_property='value')]
)
def generate_vaccination_ratio_graph(n):
    max_date = df_covid['date'].max()
    df = df_covid[df_covid['date'] == max_date]
    df['vaccination_ratio'] = df['total_vaccinations'] / df['population']
    df = df.sort_values(by='vaccination_ratio', ascending=False).head(n).fillna(0)
    fig = px.bar(
        data_frame=df,
        x='location',
        y='vaccination_ratio',
        title=f'Vaccination ratio',
        labels={'vaccination_ratio': 'Vaccination ratio'}
    )
    return fig


In [ ]:
##aplikaci spouštím pomocí threading knihovny a output modulu z google.colabknihovny
thread = threading.Thread(target=third_dash_app.run_server)
thread.start()
output.serve_kernel_port_as_window(port=8050)
